<a href="https://colab.research.google.com/github/talhahk24/Data_Scraping_Compilation/blob/main/(November_2023)_DataScraping_Tasks_Socif_Limited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 22.4 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,086 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy In

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
import time
import sys
from selenium.webdriver.support.ui import Select

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


driver = webdriver.Chrome(options=chrome_options)


# Rotary Club and Travel Industry Council Data Scraping

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains


driver = webdriver.Chrome(options=chrome_options)


def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

def scroll_down_page(driver, speed=8):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")



def scrape_club_data(driver):
    scroll_down_page(driver)
    club_data = {'Name': None, 'Email': None, 'Phone': None, 'Language': None}
    wait = WebDriverWait(driver, 10)

    try:
        name_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.mb-5 h1")))
        club_data['Name'] = name_element.text
    except TimeoutException:
        print("Name not found on the club page.")

    try:
        email_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h5[contains(text(), 'Email')]/following-sibling::span")))
        club_data['Email'] = email_element.text
    except TimeoutException:
        print("Email not found on the club page.")

    try:
        phone_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h5[contains(text(), 'Phone')]/following-sibling::span")))
        club_data['Phone'] = phone_element.text
    except TimeoutException:
        print("Phone not found on the club page.")

    try:
        language_element = wait.until(EC.presence_of_element_located((By.XPATH, "//h5[contains(text(), 'Rotary language')]/following-sibling::span")))
        club_data['Language'] = language_element.text
    except TimeoutException:
        print("Language not found on the club page.")

    return club_data if any(club_data.values()) else None

all_club_data = []

driver.get("https://my.rotary.org/en/club-search?meetingLocationRange=25&clubName=&distance=25&meetingLocation=Hong%20Kong&meetingLocationLatitude=22.3193039&meetingLocationLongitude=114.1693611")

for page_num in range(1, 13):  # There are 12 pages
    print(f"Processing page {page_num}...")
    try:
        wait = WebDriverWait(driver, 20)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "ul#search-results-list li a[role='link']")))

        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

        club_links = driver.find_elements(By.CSS_SELECTOR, "ul#search-results-list li a[role='link']")

        for link in club_links:
            href = link.get_attribute('href')
            driver.execute_script(f"window.open('{href}');")
            driver.switch_to.window(driver.window_handles[1])
            club_data = scrape_club_data(driver)
            if club_data:
                all_club_data.append(club_data)

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

        if page_num < 12:
            next_page = driver.find_element(By.XPATH, f"//option[@value='{page_num + 1}']")
            scroll_to_element(next_page)
            next_page.click()
    except TimeoutException:
        print(f"Timeout occurred on page {page_num}. Moving to the next page.")

    print(f"Data collected so far (after page {page_num}):")
    for club in all_club_data:
            print(club)
driver.quit()

for club in all_club_data:
    print(club)


In [ ]:
import csv

fieldnames = ['Name', 'Email', 'Phone', 'Language']

with open('club_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for club in all_club_data:
        if club is not None:
            writer.writerow(club)

print("Data exported to 'club_data.csv'")


# Travel Industry  Council of Hong Kong Data Extraction


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import csv


def scroll_down_page(driver, speed=8):
    current_scroll_position, new_height = 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")


def scrape_agent_data(driver):
    scroll_down_page(driver)
    agent_data = {'Name': None, 'Phone': None, 'Email': None}
    wait = WebDriverWait(driver, 10)

    try:
        name_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ta-title-en")))
        if name_element.text.strip() == "":
            name_element = driver.find_element(By.CSS_SELECTOR, ".ta-title-chi")
        agent_data['Name'] = name_element.text.strip()
    except TimeoutException:
        print("Name not found on the agent page.")

    try:
        phone_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='tel']")))
        agent_data['Phone'] = phone_element.text
    except TimeoutException:
        print("Phone not found on the agent page.")

    try:
        email_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='mailto']")))
        agent_data['Email'] = email_element.text
    except TimeoutException:
        print("Email not found on the agent page.")

    return agent_data if any(agent_data.values()) else None

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

all_agent_data = []

for page_num in range(0, 105):
    print(f"Processing page {page_num}...")
    driver.get(f"https://tichk.org/zh-hant/agents/find-agent?combine=&find-ta-location=&page={page_num}")
    wait = WebDriverWait(driver, 10)

    agent_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.ta-details-rows a")))

    for link in agent_links:
        href = link.get_attribute('href')
        driver.execute_script(f"window.open('{href}');")
        driver.switch_to.window(driver.window_handles[1])
        agent_data = scrape_agent_data(driver)
        if agent_data:
            all_agent_data.append(agent_data)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])

    print(f"Data collected so far (after page {page_num}):")
    for agent in all_agent_data:
        print(agent)

driver.quit()

with open('agent_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['Name', 'Phone', 'Email'])
    writer.writeheader()
    for agent in all_agent_data:
        writer.writerow(agent)

print("Data exported to 'agent_data.csv'")


***Its taking too long so I will be removing scrolling and decrreasing wait time***

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import csv

def scroll_down_page(driver, speed=8):
    current_scroll_position, new_height = 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")

def scrape_agent_data(driver):
    agent_data = {'Name': None, 'Phone': None, 'Email': None}
    wait = WebDriverWait(driver, 5)  # Reduced wait time to 5 seconds

    try:
        name_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".ta-title-en")))
        if name_element.text.strip() == "":
            name_element = driver.find_element(By.CSS_SELECTOR, ".ta-title-chi")
        agent_data['Name'] = name_element.text.strip()
    except TimeoutException:
        print("Name not found on the agent page.")

    try:
        phone_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='tel']")))
        agent_data['Phone'] = phone_element.text
    except TimeoutException:
        print("Phone not found on the agent page.")

    try:
        email_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[href^='mailto']")))
        agent_data['Email'] = email_element.text
    except TimeoutException:
        print("Email not found on the agent page.")

    return agent_data if any(agent_data.values()) else None

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=chrome_options)

all_agent_data = []

for page_num in range(0, 105):
    print(f"Processing page {page_num}...")
    driver.get(f"https://tichk.org/zh-hant/agents/find-agent?combine=&find-ta-location=&page={page_num}")
    wait = WebDriverWait(driver, 10)

    agent_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.ta-details-rows a")))
    hrefs = [link.get_attribute('href') for link in agent_links]

    for href in hrefs:
        driver.get(href)
        agent_data = scrape_agent_data(driver)
        if agent_data:
            all_agent_data.append(agent_data)

    print(f"Data collected so far (after page {page_num}):")
    for agent in all_agent_data:
        print(agent)

driver.quit()

with open('agent_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['Name', 'Phone', 'Email'])
    writer.writeheader()
    for agent in all_agent_data:
        writer.writerow(agent)

print("Data exported to 'agent_data.csv'")

# Home Affairs Department Data Extraction

In [11]:
driver.save_screenshot("/content/startup.png")
print(f"Screenshot saved to {screenshot_path}")

initial_url = "https://bmis2.buildingmgt.gov.hk/bd_hadbiex/content/searchbuilding/building_search.jsf?renderedValue=true"
driver.get(initial_url)

Screenshot saved to /content/screenshot_debug.png


Change Language

In [ ]:
Lannguage_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_id_s:j_id_1l")))
ActionChains(driver).move_to_element(Lannguage_button).click(Lannguage_button).perform()
time.sleep(10)

driver.save_screenshot("/content/screenshot_after_changing_language.png" )
print(f"Screenshot saved to {screenshot_path}")

Switch 10 entries per page to 20 entries per page

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)

dropdown_element = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.ID, "_bld_result_frm:_result_tbl:j_id__v_6"))
)
select = Select(dropdown_element)
select.select_by_value("20")

time.sleep(10)

driver.save_screenshot("/content/screenshot_after_changing_10_to_20.png"  )
print(f"Screenshot saved to {screenshot_path}")


Screenshot saved to /content/sample_data/screenshot_after_changing_10_to_20.png


Mount Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv

def safe_get_text(element):
  try:
      return element.text
  except:
      return "None"


checkpoints= [ i for i in range(3000, 42000, 3000)]
screenshot_path = "/content/sample_data/screenshot_after_click_next_latest.png"
csv_file_path = "/content/drive/MyDrive/buildings_data.csv"


# 7min avg per checkpoint
for checkpoint in checkpoints:
    while b_num <= checkpoint:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        total_rows = len(driver.find_elements(By.CSS_SELECTOR, "tbody#_bld_result_frm\\:_result_tbl_data > tr"))
        current_index = 0
        while current_index < total_rows:
            rows = driver.find_elements(By.CSS_SELECTOR, "tbody#_bld_result_frm\\:_result_tbl_data > tr")
            row = rows[current_index]
            elements = row.find_elements(By.CSS_SELECTOR, "td[role='gridcell']")
            name_element = safe_get_text(elements[0])
            org_element = safe_get_text(elements[1])
            owner_element = safe_get_text(elements[2])

            Buildings_data["Names"].append(name_element)
            Buildings_data["Organisations"].append(org_element)
            Buildings_data["Incorporated Owners"].append(owner_element)
            if b_num == 0 or b_num == 1:
                print(Buildings_data)
            print(b_num)
            b_num += 1
            current_index += 1
        try:

            Next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".ui-paginator-next.ui-state-default.ui-corner-all")))
            ActionChains(driver).move_to_element(Next_button).click(Next_button).perform()
            driver.save_screenshot(screenshot_path)
        except:

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            ActionChains(driver).move_to_element(Next_button).click(Next_button).perform()
            driver.save_screenshot(screenshot_path)

        l_num = l_num + 20
        time.sleep(5)
        driver.save_screenshot(screenshot_path)
    print(f"current {b_num}")
    print(f"current {f_num}")
    print(f"checkpoint reached {checkpoint}")

    with open(csv_file_path, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Organisation', 'Incorporated Owner'])
        for i in range(len(Buildings_data['Names'])):
            writer.writerow([Buildings_data['Names'][i], Buildings_data['Organisations'][i], Buildings_data['Incorporated Owners'][i]])

    print(f"CSV file has been saved to {csv_file_path}")
